In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os 
from skimage import io
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from Models import *
from SDMUtils import *

In [ ]:
trial = 1

# Choose architecture 
architecture = 'ViViT'  # 'ViViT' or 'LSTM' or 'VGG16'

# Choose number of timepoints, planes 
timespace = 1
plane=1

# Choose a name for the output directory
folder = f"{architecture}_{datetime.today().strftime('%Y%m%d')}_{timespace}tstep_{plane}plane_Fluo_" + str(trial)

# DO NOT MODIFY FOR REPRODUCIBILITY
seed= 12

In [ ]:
############################# Check inconsistency ##################################
try :
    if architecture=='VGG16' and timespace>1 :
        raise Error("VGG16 accepts 1 timepoint only")
        
except Error as e:
    print(e)

In [ ]:
os.mkdir(folder)

## Load images 

In [ ]:
# Current directory 
directory = os.getcwd()
print(directory)

# Load Events 
folders = os.walk(directory+'/data/Events')
data = [x[0] for x in folders]
data = data[1:]

events=[]
for i in tqdm(range(len(data))):
    events+= [np.transpose(np.reshape([(
        io.imread(data[i]+'/'+f)-np.min(io.imread(data[i]+'/'+f)))/(np.max(io.imread(data[i]+'/'+f))-np.min(io.imread(data[i]+'/'+f))) # normalize pixels between 0 and 1
        for f in sorted(os.listdir(data[i]))[-8*timespace:]],  # take the 8 planes of the last t timepoints
        (-1,8,352,352)), # reshape to split planes and timepoints ; shape=(timepoints,planes,im_shape,im_shape)
        (0,2,3,1)) # reshape to have (timepoints,im_shape,im_shape,plane)
        ] 
    
# Load Non-Events
ne = os.walk(directory+'/data/Non-Events')
data = [x[0] for x in ne]
non_events=[]
data = data[1:]
for i in tqdm(range(len(data))):
    non_events+= [np.transpose(np.reshape([(
        io.imread(data[i]+'/'+f)-np.min(io.imread(data[i]+'/'+f)))/(np.max(io.imread(data[i]+'/'+f))-np.min(io.imread(data[i]+'/'+f))) # normalize pixels between 0 and 1
        for f in sorted(os.listdir(data[i]))[-8*timespace:]],  # take the 8 planes of the last t timepoints
        (-1,8,352,352)), # reshape to split planes and timepoints ; shape=(timepoints,planes,im_shape,im_shape)
        (0,2,3,1)) # reshape to have (timepoints,im_shape,im_shape,plane)
        ] 
    

## Preprocess data

In [ ]:
try : 
    x_train, y_train, x_test, y_test = prepare_sets(events,non_events,seed,timespace,plane)            
except Error as e:
    print(e)

In [ ]:
del data 
del events,non_events
del directory, folders, ne

## Train the model

In [ ]:
try : 
    acc_lw,hist=train(x_train,y_train,x_test,y_test,seed,architecture,timespace,plane,folder)
except Error as e:
    print(e)